<a href="https://colab.research.google.com/github/Heba-Ajami/Applied-Data-Science-Capstone/blob/main/Syrian%20Parliament%20Election%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()   # select all cleaned Excel files

Saving Aleppo - Final.xlsx to Aleppo - Final (2).xlsx
Saving Aleppo - First.xlsx to Aleppo - First (2).xlsx
Saving Candidates.xlsx to Candidates (2).xlsx
Saving Damascus - Final.xlsx to Damascus - Final (2).xlsx
Saving Damascus - First.xlsx to Damascus - First (2).xlsx
Saving Daraa - Final.xlsx to Daraa - Final (2).xlsx
Saving Daraa - First.xlsx to Daraa - First (2).xlsx
Saving Deir - Final.xlsx to Deir - Final (2).xlsx
Saving Deir - First.xlsx to Deir - First (2).xlsx
Saving Hama - Final.xlsx to Hama - Final (2).xlsx
Saving Hama - First.xlsx to Hama - First (2).xlsx
Saving Homs - Final.xlsx to Homs - Final (2).xlsx
Saving Homs - First.xlsx to Homs - First (2).xlsx
Saving Idlib - Final.xlsx to Idlib - Final (2).xlsx
Saving Idlib - First.xlsx to Idlib - First (2).xlsx
Saving Latikkia - Final.xlsx to Latikkia - Final (2).xlsx
Saving Latikkia - First.xlsx to Latikkia - First (2).xlsx
Saving Qunaitera - Final.xlsx to Qunaitera - Final (2).xlsx
Saving Qunitera - First.xlsx to Qunitera - Fir

In [ ]:
df = pd.read_excel(list(uploaded.keys())[0])
print(df.columns)

Index(['القائمة', 'الدائرة الانتخابية', 'المدينة', 'المواليد',
       'الاسم الثلاثي'],
      dtype='object')


In [ ]:
import pandas as pd
import re

all_dfs = []

for filename in uploaded.keys():
    df = pd.read_excel(filename)
    df.columns = df.columns.str.strip()

    # أسماء الأعمدة العربية
    rename_map = {
        "القائمة": "list_type",
        "الدائرة الانتخابية": "district",
        "المدينة": "city",
        "المواليد": "birth_info",
        "الاسم الثلاثي": "name"
    }
    df = df.rename(columns={c: rename_map.get(c, c) for c in df.columns})

    for col in ["name", "city", "district", "birth_info", "list_type"]:
        if col not in df.columns:
            df[col] = None

    # استخراج سنة الميلاد
    df["birth_year"] = df["birth_info"].astype(str).str.extract(r"(\d{4})")
    df["birth_year"] = df["birth_year"].fillna("").astype(str)

    # استخراج مكان الميلاد
    df["birthplace"] = (
        df["birth_info"].astype(str)
        .str.replace(r"\d{4}", "", regex=True)
        .str.replace("/", "")
        .str.strip()
    )

    for col in ["name", "city", "district"]:
        df[col] = df[col].astype(str).str.strip()

    # الأعلام (Flags)
    df["in_first"] = df["list_type"].str.contains("الأولى|first", case=False, na=False).astype(int)
    df["in_final"] = df["list_type"].str.contains("النهائية|final", case=False, na=False).astype(int)
    df["is_candidate"] = df["list_type"].str.contains("مرشح|candidate", case=False, na=False).astype(int)

    keep_cols = [
        "name", "birth_year", "city", "district", "birthplace",
        "in_first", "in_final", "is_candidate"
    ]
    df = df[keep_cols]
    all_dfs.append(df)

# دمج جميع الملفات
combined_raw = pd.concat(all_dfs, ignore_index=True)

# دالة لأخذ أول قيمة غير فارغة
def first_valid(series):
    return series.dropna().iloc[0] if not series.dropna().empty else None

# الدمج فقط بالاسم + سنة الميلاد
combined = (
    combined_raw
    .groupby(["name", "birth_year"], as_index=False)
    .agg({
        "city": first_valid,
        "district": first_valid,
        "birthplace": first_valid,
        "in_first": "max",
        "in_final": "max",
        "is_candidate": "max"
    })
)

print("✅ الشكل النهائي:", combined.shape)
combined.head(20)

✅ الشكل النهائي: (7683, 8)


,name,birth_year,city,district,birthplace,in_first,in_final,is_candidate
0,(أحمد عبد الحمود (الجعفر,1965,الرقة,الرقة,اﻟرﺣﺑﻲ,1,0,0
1,(أحمد محمود الخلف (الشش,1980,الرقة,الرقة,ﻣﻌدان,1,0,0
2,(إبراھم عبد العبد (إسبر,1978,حماه,مصياف,ﻣﺻﯾﺎف,1,1,0
3,(عبد الستار عسى احمد (القرا,1988,الرقة,الرقة,ﺷرﻛراك,1,0,0
4,(عصام عبد القادر المحمد ( الحاج عبو,1973,الرقة,الرقة,اﻟرﻗﺔ,1,0,0
5,.رغد عبد الرزاق حلمي,1985,دمشق,دمشق,دﻣﺷق,1,0,0
6,nan,,nan,nan,nan,0,0,0
7,آء إبراھم العلي,1998,حمص,تلكلخ,ﺗﻠﻛﻠﺦ,1,1,0
8,آء بدر الدن المحامد,1988,درعا,None,درعا البلد,1,1,0
9,آء تسر الفاعوري,1990,درعا,None,الشخ مسكن,1,1,0


In [ ]:
combined.to_excel("voter_master.xlsx", index=False)
from google.colab import files
files.download("voter_master.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>